In [ ]:
#### add width, height and aspect ratio columns to dataframe #####

width, heights, ratio = [], [], []

for path in tqdm(df_train["filename"], desc = "Analyzing image sizes"):
    try:
        with Image.open(os.path.join(train_img, path)) as img:
            w, h = img.size
            width.append(w)
            heights.append(h)
            ratio.append(w / h)
    except:
        print(f"Image {os.path.join(train_img, path)} cant be opened adding null data")
        widths.append(None)
        heights.append(None)
        ratios.append(None)
df_train["width"] = width
df_train["heights"] = heights
df_train["ratio"] = ratio

df_train[['width', 'heights', 'ratio']].head()

In [ ]:
#### Stats Per Class ####
def image_stats(df, sample_size = 200, image_col="filename", data_dir = None, per_class = False, label_col = 'genus'):
    sample_df = df.sample(min(sample_size, len(df))).reset_index(drop=True)
    mean_rgb, std_rgb = [], []
    per_class_stats = {}

    for _, row in tqdm (sample_df.iterrows(), total = len(sample_df), desc="computing RGB stats"):
        path = row[image_col]
        label = row[label_col] if label_col in row else unknown
        if data_dir is not None:
            path = os.path.join(data_dir, path)
        img = cv2.imread(path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0

        mean = img.mean(axis=(0, 1))
        std = img.std(axis=(0, 1))

        mean_rgb.append(mean)
        std_rgb.append(std)

        if per_class:
            if label not in per_class_stats:
                per_class_stats[label] = {"mean": [], "std": []}
            per_class_stats[label]["mean"].append(mean)
            per_class_stats[label]["std"].append(std)

    mean_rgb = np.mean(mean_rgb, axis = 0)
    std_rgb = np.mean(std_rgb, axis = 0)

    if per_class:
        for cls in per_class_stats:
            per_class_stats[cls]["mean"] = np.mean(per_class_stats[cls]["mean"], axis = 0)
            per_class_stats[cls]["std"] = np.mean(per_class_stats[cls]["std"], axis = 0)
        return mean_rgb, std_rgb, per_class_stats
    else:
        return mean_rgb, std_rgb

In [ ]:
mean_rgb, std_rgb, per_class_stats = image_stats(df_train, data_dir = train_img, per_class = True)

print(f"Global mean RGB: {mean_rgb}")
print(f"Global std RGB: {std_rgb}")
for cls, stats in per_class_stats.items():
    print(f"{cls}: Mean {stats["mean"]}, Std {stats["std"]}")   